In [17]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

## TCGA data

In [4]:
TCGA = pd.read_csv('TCGA_data.csv')
TCGA = TCGA.loc[(TCGA.cancer == 'BLCA')]
TCGA.loc[(TCGA.cancer == 'BLCA') & (TCGA.y != 'Normal'),'y'] = 'BC'
TCGA.y.value_counts()

BC        408
Normal    375
Name: y, dtype: int64

## urine data

In [5]:
BC_100 = pd.read_csv('BC_32ea_k100_cyto.csv')
Normal_100 = pd.read_csv('Normal_21ea_k100_cyto.csv')

#drop person id
BC_100.drop(['Unnamed: 0','id'], axis = 1, inplace = True)
Normal_100.drop(['Unnamed: 0','id'], axis = 1, inplace = True)

BC_100['cancer'] = 'BC'
Normal_100['cancer'] = 'normal'

K100 = pd.concat([BC_100,Normal_100], axis = 0)

del [BC_100,Normal_100]

K100 = K100.dropna(axis=0)
K100.isna().sum().sum()

0

## 공통 col 추출

In [6]:
a = list(TCGA.columns)
b = list(K100.columns)

new_col = []
for col in a :
    for col2 in b :
        if col==col2 :
            new_col.append(col)       

In [7]:
TCGA_new = TCGA[new_col]
K100_new = K100[new_col]

## 1. scale : 기본 스케일. 평균과 표준편차 사용

In [37]:
# modeling
X = TCGA_new.drop(['cancer'], axis=1)
X = scale(X)
Y = TCGA['y']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3)

rf = RandomForestClassifier()
rf.fit(X_train,y_train)

predicted = rf.predict(X_test)
accuracy1 = accuracy_score(y_test, predicted)

X = K100_new.drop(['cancer'], axis=1)
X = scale(X)
Y = K100['cancer']

predicted = rf.predict(X)
accuracy11 = accuracy_score(Y, predicted)

## 2. robust_scale

In [32]:
# modeling
X = TCGA_new.drop(['cancer'], axis=1)
X = robust_scale(X)
Y = TCGA['y']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3)

rf = RandomForestClassifier()
rf.fit(X_train,y_train)

predicted = rf.predict(X_test)
accuracy2 = accuracy_score(y_test, predicted)

# test
X = K100_new.drop(['cancer'], axis=1)
X = robust_scale(X)
Y = K100['cancer']

predicted = rf.predict(X)
accuracy22 = accuracy_score(Y, predicted)

## 3. minmax_scale

In [34]:
# modeling
X = TCGA_new.drop(['cancer'], axis=1)
X = minmax_scale(X)
Y = TCGA['y']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3)

rf = RandomForestClassifier()
rf.fit(X_train,y_train)

predicted = rf.predict(X_test)
accuracy3 = accuracy_score(y_test, predicted)

# test
X = K100_new.drop(['cancer'], axis=1)
X = minmax_scale(X)
Y = K100['cancer']

predicted = rf.predict(X)
accuracy33 = accuracy_score(Y, predicted)

## 4. maxabs_scale

In [41]:
# modeling
X = TCGA_new.drop(['cancer'], axis=1)
X = maxabs_scale(X)
Y = TCGA['y']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3)

rf = RandomForestClassifier()
rf.fit(X_train,y_train)

predicted = rf.predict(X_test)
accuracy4 = accuracy_score(y_test, predicted)



In [42]:
# test
X = K100_new.drop(['cancer'], axis=1)
X = maxabs_scale(X)
Y = K100['cancer']

predicted = rf.predict(X)
accuracy44 = accuracy_score(Y, predicted)

In [43]:
pd.DataFrame({'scale':['scale','robust','minmax','maxabs'],'train':[accuracy1,accuracy2,accuracy3,accuracy4],
             'test':[accuracy11,accuracy22,accuracy33,accuracy44]})

,scale,train,test
0,scale,0.978723,0.603774
1,robust,0.965957,0.566038
2,minmax,0.978723,0.603774
3,maxabs,0.961702,0.603774
